앙상블

앙상블 학습은 여러 대의 예측기의 결과를 수집하여 종합한 후 예측값을 정하는 학습법이다.
성능이 약한 학습기 여러 대를 앙상블 학습에 이용하면 보다 강력한 학습기를 생성할 수 있다.
전제 조건은 아래와 같다.
- 학습기 사이의 독립성과 다양성이 보장되어야 한다.
- 충부히 많은 학습기가 사용되어야 한다.

큰 수의 법칙
- 51%의 확률로 앞면이 나오는 동전 1만번 던지기를 통해 설명한다.
    - 동전 1만번 던지기를 10번 실행한 결과는 모양의 넘파이 어레이를 무작위로 생성된 0과 1사이의수를 이용하는 것으로 대신한다.
    - 어레이의 각 항목을 0.51보다 작으면 True(앞면)으로 간주한다.

In [ ]:
import numpy as np
# 앞면이 나올 확률
heads_proba = 0.51

# 동전 1만번 던지기를 10번 실행한 결과, 0.51보다 작으면 True 즉, 앞면
coin_tosses = (np.random.rand(10000, 10) < heads_proba).astype(np.int32)

# 1만번 던질 때까지 각 단계에서 앞면이 나올 비율
cumulative_heads_ratio = np.cumsum(coin_tosses, axis=0) / np.arange(1, 10001).reshape(-1, 1)

아래 코드는 1만 번 던지기를 10번 실행할 때 앞면이 나올 확률이 모두 50% 이상인 값으로 수렴하는 사실을 보여준다.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 3.5))

# 10번의 모의실험 결과 그래프
plt.plot(cumulative_heads_ratio)
# 51% 수렴선
plt.plot([0, 10000], [0.51, 0.51], "k--", linewidth=2, label="51%")
# 50% 경계선
plt.plot([0, 10000], [0.5, 0.5], "k-", label="50%")

plt.xlabel("Number of coin tosses")
plt.ylabel("Heads ratio")
plt.legend(loc="lower right")
plt.axis([0, 10000, 0.42, 0.58])

plt.show()

ensemble_win_proba() 함수를 이용하여 정확도 p의 예측기 n개를 실행하여 다수결 원칙을 따랐을 때의 승률 즉, 다수결의 의견이 옳았을 확률을 계산하다.
- 이를 위해 이항분포의 누적밀도함수인 binom.cdf()를 이용한다.

binom.cdf(x, n, p) 함수는 이항분포를 따르는 어떤 행위를 n번 시행했을 때 확률 p의 사건이 x번 이하 발생할 확률을 계산하는 누적분포함수이다.
아래 코드에서는 50%이상일 확률을 계산해야 하기에, x대신 50%보다 아주 조금 작은 빈도수를 입력하여 얻은 확률을 1에서 뺀다.

In [ ]:
from scipy.stats import binom

def ensemble_win_proba(n, p):
    """
    p: 예측기 하나의 성능
    n: 앙상블 크기 즉, 예측기의 개수
    반환값: 다수결을 따를 떄 성공할 확률, 이항 분포의 누적분포함수 활용
    """
    return 1 - binom.cdf(int(n*0.499999), n, p)

51%의 확률로 앞면이 나오는 동전을 1천 번 던져서 절반 이상 앞면이 나올 확률은 74.7%이다.

In [ ]:
ensemble_win_proba(1000, 0.51)

횟수를 1만 번으로 늘리면 97.7%가 나온다.
- 즉, 1만 번 던지기를 100번하면 2번 정도만 5천 번 이하로 앞면이 나올 것으로 기대한다.

In [ ]:
ensemble_win_proba(10000, 0.51)

정확도가 80% 정도의 이진분류기 10대를 이용한 앙상블 학습기의 성능이 99.3%까지 오를 수 있다.
- 실전에서는 최적으로 튜닝된 모델의 성능을 앙상블 학습을 통해 개선하는 시도를 많이 한다.
- 앙상블은 사용된 예측기의 다양성과 상호 독립성이 전제도니 경우에 가능하다.

In [ ]:
ensemble_win_proba(10, 0.8)

직접 투표 vs 간접 투표

아래 코드는 여러 종류의 분류기를 앙상블로 묶어 훈련시키는 방법이다.

사용되는 훈련데이터는 moons 데이터셋이다.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

VotingClassifier 모델은 여러개의 분류기를 이용한 앙상블 학습 분류기를 지원한다.
사용되는 분류기는 아래의 3개이다.
- 로지스틱 회귀
- 랜덤 포레스트 분류기
- SVM

직접 투표

voting="hard"는 분류기들의 다수 예측값을 예측값으로 사요ㅕㅇ한다는 의미이다.
- 이러한 방식을 직접 투표(hard voting)라고 한다.

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

voting_clf = VotingClassifier(
    estimators=[("lr", log_clf), ("rf", rnd_clf), ("svm", svm_clf)], voting="hard"
)

voting_clf.fit(X_train, y_train)

앙상블 학습에 사용된 분류기 3개에 비해 분류기의 정확도 성능이 좋은 것을 아래 코드에서 확인할 수 있다.

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

간접 투표

주어진 샘플의 클래스 확률을 계산하는 분류기들을 앙상블 학습에 사용하고, voting="soft"로 지정하면 분류기들의 클래스 확률의 평균값을 이용한다.
- SVM의 경우 probability=True 옵션을 지정해야 클래스 확률을 사용할 수 있다.
- 클래스 확률은 predict_proba() 메서드가 계산한다.

In [ ]:
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", probability=True, random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting='soft'
)

voting_clf.fit(X_train, y_train)

간접 투표 방식이 직접 투표 방식보다 정확도를 보다 높인다.
- 하지만 기본값은 직접 투표이다.

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

배깅과 페이스팅

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42),
    n_estimators=500,
    max_samples=100,
    bootstrap=True,
    random_state=42
)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree))

아래 코드는 결정트리 모델 하나를 규제없이 사용할 때와 결정트리 500개를 사용하는 배깅 분류기의 성능을 비교하는 그래프를 그린다.

In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[-1.5, 2.45, -1, 1.5], alpha=0.5, contour=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap=ListedColormap(["#fafab0", "#9898ff", "#507d50"])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if contour:
        custom_cmap2 = ListedColormap(["#7d7d58", "#4c4c7f", "#507d50"])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", alpha=alpha)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", alpha=alpha)
    plt.axis(axes)
    plt.xlabel(r"$x_1$", fontsize=18)
    plt.ylabel(r"$x_2$", fontsize=18, rotation=0)

In [ ]:
fix, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)

# 왼쪽 그림
plt.sca(axes[0])
plot_decision_boundary(tree_clf, X, y)
plt.title("Decision Tree", fontsize=14)

# 오른쪽 그림
plt.sca(axes[1])
plot_decision_boundary(bag_clf, X, y)
plt.title("Decision Trees with Bagging", fontsize=14)

plt.ylabel("")
plt.show()

아래 코드는 훈련 후 oob 평가를 진행한 결과를 보여준다.

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42),
    n_estimators=500,
    bootstrap=True,
    oob_score=True,
    random_state=40
)

bag_clf.fit(X_train, y_train)

print(bag_clf.oob_score_)

테스트 성능과 유사한 결과이다.

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

oob_decision_function_ 속성은 훈련 샘플의 클래스 확률을 저장한다.
- 처음 10개 훈련 샘플의 클래스 확률은 아래와 같다.

In [ ]:
bag_clf.oob_decision_function_[:10]

랜덤 포레스트
- 랜덤 포레스트는 기본적으로 결정트리를 이용한 배깅 모델을 최적화한 결과이다.

In [ ]:
# 결정 트리를 이용한 배깅 모델
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(
        splitter="random",
        max_leaf_nodes=16,
        random_state=42
    ),
    n_estimators=500,
    max_samples=1.0,
    bootstrap=True,
    random_state=42
)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [ ]:
# 랜덤 포레스트 모델
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(
    n_estimators=500,
    max_leaf_nodes=16,
    random_state=42
)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

두 모델의 예측값의 97.6% 정도로 동일하다.

In [ ]:
np.sum(y_pred == y_pred_rf) / len(y_pred)

아래 코드는 배깅 기법이 작동하는 방식을 보여준다.
- 결정트리 모델을 독립적으로 15번 훈련
- max_samples=1.0, bootstrap=True 옵션 처럼 샘플 중복 선택 허용
- 15개의 모델에 대해 결정경계를 희미하게 보여준다.
    - 배깅 모델의 경우 분류는 최빈값, 회귀는 평균값을 사용한다.

In [ ]:
plt.figure(figsize=(6, 4))

for i in range(15):
    tree_clf = DecisionTreeClassifier(max_leaf_nodes=16, random_state=42 + i)
    indices_with_replacement = np.random.randint(0, len(X_train), len(X_train))
    tree_clf.fit(X[indices_with_replacement], y[indices_with_replacement])
    plot_decision_boundary(tree_clf, X, y, axes=[-1.5, 2.45, -1, 1.5], alpha=0.02, contour=False)

plt.show()

특성 중요도

선택된 특성과 임계값의 중요도를 훈련과정을 통해 측정한다.

붓꽃 특성 중요도
- 꽃잎의 길이와 너비가 가장 중요하다.

In [ ]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

MNIST 이미지 특성 중요도

아래 코드는 MNIST 사진에 포함된 픽셀별 중요도를 이미지로 보여준다.

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml("mnist_784", version=1)
mnist.target = mnist.target.astype(np.uint8)

rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rnd_clf.fit(mnist["data"], mnist["target"])

In [ ]:
import matplotlib as mpl
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap=mpl.cm.hot, interpolation="nearest")
    plt.axis("off")

기본적으로 숫자가 쓰여진 사진 중심부의 큰 부분이 중요함을 아래 그림에서 확인할 수 있다.

In [ ]:
plot_digit(rnd_clf.feature_importances_)

cbar = plt.colorbar(
    ticks=[rnd_clf.feature_importances_.min(),
           rnd_clf.feature_importances_.max()]
)
cbar.ax.set_yticklabels(["Not important", "Very important"])

plt.show()

에이다부스트

In [ ]:
m = len(X_train)

fix, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
for subplot, learning_rate in ((0, 1), (1, 0.5)):
    sample_weights = np.ones(m) / m
    plt.sca(axes[subplot])
    for i in range(5):
        svm_clf = SVC(kernel="rbf", C=0.2, gamma=0.6, random_state=42)
        svm_clf.fit(X_train, y_train, sample_weight=sample_weights * m) # 업데이트된 샘플 가중치 사용
        y_pred = svm_clf.predict(X_train)

        r = sample_weights[y_pred != y_train].sum() / sample_weights.sum() # 7-1 식
        alpha = learning_rate * np.log((1-r)/r) # 7-2 식
        sample_weights[y_pred != y_train] *= np.exp(alpha) # 7-3 식
        sample_weights /= sample_weights.sum() # normalization step

        plot_decision_boundary(svm_clf, X, y, alpha=0.2)
        plt.title("learning_rate = {}".format(learning_rate), fontsize=16)
    if subplot == 0:
        plt.text(-0.75, -0.95, "1", fontsize=14)
        plt.text(-1.05, -0.95, "2", fontsize=14)
        plt.text(1.0, -0.95, "3", fontsize=14)
        plt.text(-1.45, -0.5, "4", fontsize=14)
        plt.text(1.36,  -0.95, "5", fontsize=14)
    else:
        plt.ylabel("")

plt.show()

사이킷런의 AdaBoostClassifier 모델 활용
- AdaBoostClassifier의 기본 모델은 다음과 같다.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth= 1),
    n_estimators=200,
    algorithm="SAMME.R",
    learning_rate=0.5,
    random_state=42
)

moons 데이터셋에 훈련시킨 결과는 다음과 같다.

In [ ]:
ada_clf.fit(X_train, y_train)
plot_decision_boundary(ada_clf, X, y)

그레이디언트 부스팅

아래 코드는 잡음이 섞인 2차 함수 그래프 형태의 훈련 세트를 사용하여 그레이디언트 부스팅 기법을 회귀 모델 훈련에 적용하는 것을 보여준다.

In [ ]:
np.random.seed(42)
X=np.random.rand(100, 1)-0.5
y=3*X[:, 0]**2 + 0.05 * np.random.randn(100)

결정트리 모델을 훈련시킨다.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

잔여오차에 대해 다시 결정트리 모델을 훈련시킨다.

In [ ]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X, y2)

한번 더 잔여오차에 대하여 다시 결정트리 모델을 훈련시킨다.

In [ ]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X, y3)

예측값은 사용된 예측기들의 예측값을 모두 더한 값이다.

In [ ]:
X_new = np.array([[0.8]])

In [ ]:
y_pred = sum(tree.predict(X_new) for tree in  (tree_reg1, tree_reg2, tree_reg3))
print(y_pred)

아래 함수는 회귀모델의 예측 성능을 그래프로 그린다.

In [ ]:
def plot_predictions(regressors, X, y, axes, label=None, style="r-", data_style="b.", data_label=None):
    x1 = np.linspace(axes[0], axes[1], 500)
    y_pred = sum(regressor.predict(x1.reshape(-1, 1)) for regressor in regressors)
    plt.plot(X[:, 0], y, data_style, label=data_label)
    plt.plot(x1, y_pred, style, linewidth=2, label=label)
    if label or data_label:
        plt.legend(loc="upper center", fontsize=16)
    plt.axis(axes)

아래 코드는 앞서 훈련시킨 세 개의 모델의 예측 성능을 그래프로 그린다.
- 좌변: 세 개의 예측기 각각의 예측 그래프
- 우변: 누적 예측 그래프

In [ ]:
plt.figure(figsize=(11,11))

plt.subplot(321)
plot_predictions([tree_reg1], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="$h_1(x_1)$", style="g-", data_label="Training set")
plt.ylabel("$y$", fontsize=16, rotation=0)
plt.title("Residuals and tree predictions", fontsize=16)

plt.subplot(322)
plot_predictions([tree_reg1], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="$h(x_1) = h_1(x_1)$", data_label="Training set")
plt.ylabel("$y$", fontsize=16, rotation=0)
plt.title("Ensemble predictions", fontsize=16)

plt.subplot(323)
plot_predictions([tree_reg2], X, y2, axes=[-0.5, 0.5, -0.5, 0.5], label="$h_2(x_1)$", style="g-", data_style="k+", data_label="Residuals")
plt.ylabel("$y - h_1(x_1)$", fontsize=16)

plt.subplot(324)
plot_predictions([tree_reg1, tree_reg2], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="$h(x_1) = h_1(x_1) + h_2(x_1)$")
plt.ylabel("$y$", fontsize=16, rotation=0)

plt.subplot(325)
plot_predictions([tree_reg3], X, y3, axes=[-0.5, 0.5, -0.5, 0.5], label="$h_3(x_1)$", style="g-", data_style="k+")
plt.ylabel("$y - h_1(x_1) - h_2(x_1)$", fontsize=16)
plt.xlabel("$x_1$", fontsize=16)

plt.subplot(326)
plot_predictions([tree_reg1, tree_reg2, tree_reg3], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="$h(x_1) = h_1(x_1) + h_2(x_1) + h_3(x_1)$")
plt.xlabel("$x_1$", fontsize=16)
plt.ylabel("$y$", fontsize=16, rotation=0)

plt.show()

사이킷런의 GBRT 모델 활용

하이퍼파라미터가 다른 2개의 GradientBoostingRegressor 모델의 성능 차이를 알아본다.
- learning_rate=1.0: 큰 학습률
- n_estimators=3: 3개의 예측기만 사용 즉, 두 번의 잔여오차 예측 모델 훈련을 실행한다.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0, random_state=42)
gbrt.fit(X, y)

- learning_rate=0.1: 작은 학습률
- n_estimators=200: 200 개의 예측기만 사용한다. 즉, 199번의 잔여오차 예측 모델 훈련을 실행한다.

In [ ]:
gbrt_slow = GradientBoostingRegressor(max_depth=2, n_estimators=200, learning_rate=0.1, random_state=42)
gbrt_slow.fit(X, y)

아래 코드는 위 두 개의 다른 두 GBRT 모델의 성능을 그래프로 그린다.
- 좌변: 큰 학습률, 3개의 예측기
- 우변: 작은 학습률, 200개의 예측기

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)

# 좌변 그래프
plt.sca(axes[0])
plot_predictions([gbrt], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="Ensemble predictions")
plt.title("learning_rate={}, n_estimators={}".format(gbrt.learning_rate, gbrt.n_estimators_), fontsize=18)
plt.xlabel("$x_1$", fontsize=16)
plt.ylabel("$y", fontsize=16, rotation=0)

# 우변 그래프
plt.sca(axes[1])
plot_predictions([gbrt_slow], X, y, axes=[-0.5, 0.5, -0.1, 0.8])
plt.title("learning_rate={}, n_estimators={}".format(gbrt_slow.learning_rate, gbrt_slow.n_estimators_))
plt.xlabel("$x_1", fontsize=16)

plt.show()

조기종료를 사용하는 그레이디언트 부스팅

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=49)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120, random_state=42)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]

bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators, random_state=42)
gbrt_best.fit(X_train, y_train)

In [ ]:
min_error = np.min(errors)

아래는 120번 반복 훈ㄹ녀을 한 후, 최적의 예측기 개수를 알아내서 다시 훈련한 결과이다.

In [ ]:
plt.figure(figsize=(10, 4))

# 왼쪽: 평균제곱근오차 그래프
plt.subplot(121)
plt.plot(errors, "b.-")
plt.plot([bst_n_estimators, bst_n_estimators], [0, min_error], "k--")
plt.plot([0, 120], [min_error, min_error], "k--")
plt.plot(bst_n_estimators, min_error, "ko")
plt.text(bst_n_estimators, min_error*1.2, "Minimum", ha="center", fontsize=14)
plt.axis([0, 120, 0, 0.01])
plt.xlabel("Number of trees")
plt.ylabel("Error", fontsize=16)
plt.title("Validation error", fontsize=14)

# 오른쪽: 최적의 예측기 개수를 이용하여 다시 훈련된 모델
plt.subplot(122)
plot_predictions([gbrt_best], X, y, axes=[-0.5, 0.5, -0.1, 0.8])
plt.title("Best model (%d trees)" % bst_n_estimators, fontsize=14)
plt.ylabel("$y$", fontsize=16, rotation=0)
plt.xlabel("$x_1$", fontsize=16)

plt.show()

아래 코드는 평균제곱근오차가 5번 이상 개선되지 않을 때 훈련을 조기종료하도록 한다.

In [ ]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True, random_state=42)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break # early stopping

61개까지의 예측기 사용

In [ ]:
print(gbrt.n_estimators_)

훈련중 얻은 평균제곱근오차의 최소값은 다음과 같다.

In [ ]:
print(f"Minimum validation MSE: {min_val_error}")

XGBoost
- 사이킷런과 다른 패키지이며, 추가로 설치해야 한다.

In [ ]:
import xgboost

if xgboost is not None:
    xgb_reg = xgboost.XGBRegressor(random_state=42)
    xgb_reg.fit(X_train, y_train)
    y_pred = xgb_reg.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred) # Not shown
    print("Validation MSE: ", val_error)

In [ ]:
if xgboost is not None: # not Shown in the book
    xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=2)
    y_pred=xgb_reg.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred) # Not Shown
    print("Validation MSE: ", val_error) # Not Shown

In [ ]:
%timeit xgboost.XGBRegressor().fit(X_train, y_train) if xgboost is not None else None

In [ ]:
%timeit GradientBoostingRegressor().fit(X_train, y_train)

스태킹

스태킹 회귀
- 사이킷런의 Stacking Regressor 모델

In [ ]:
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor

estimators = [
    ("ridge", RidgeCV()),
    ("lasso", LassoCV(random_state=42)),
    ("knr", KNeighborsRegressor(n_neighbors=20, metric="euclidean"))
]

final_estimator = GradientBoostingRegressor(n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1, random_state=42)

reg = StackingRegressor(estimators=estimators, final_estimator=final_estimator)

모델 훈련에 사용되는 데이터셋은 사이킷런의 혈당 관련 데이터이다.

In [ ]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)

print(X.shape)

10개의 특성은 다음과 같다.
- age: 만 나이
- sex: 성별
- bmi: 체닐량지수
- bp: 평균 혈압
- s1 tc: 총 콜레스테롤
- s2 ldl: 저밀도 콜레스테롤
- s3 hdl: 고밀도 콜레스테롤
- s4 tch: 총 콜레스테롤
- s5 ltg: 중성지방 수치의 로그값
- s6 glu: 혈당 수치

예측값은 1년 뒤의 당뇨병 진행도이다.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)
print(y_pred[:5])

trainform()메서드는 훈련에 사용된 예측기별 예측값을 반환한다.

In [ ]:
reg.transform(X_test[:5])

다중 스태킹 회귀

In [ ]:
from sklearn.ensemble import RandomForestRegressor

final_layer_rfr = RandomForestRegressor(
    n_estimators=10, max_features=1, max_leaf_nodes=5, random_state=42
)

final_layer_gbr = GradientBoostingRegressor(
    n_estimators=10, max_features=1, max_leaf_nodes=5, random_state=42
)
final_layer = StackingRegressor(
    estimators=[
        ("rf", final_layer_rfr),
        ("gbrt", final_layer_gbr)
    ], final_estimator=RidgeCV()
)
multi_layer_regressor = StackingRegressor(
    estimators=[
        ("ridge", RidgeCV()),
        ("lasso", LassoCV(random_state=42)),
        ("knr", KNeighborsRegressor(n_neighbors=20, metric="euclidean"))
    ],
    final_estimator=final_layer
)

In [ ]:
multi_layer_regressor.fit(X_train, y_train)

y_pred = multi_layer_regressor.predict(X_test)
y_pred[:5]

In [ ]:
multi_layer_regressor.transform(X_test[:5])

스태킹 분류

사이킷런의 StackingClassifier 모델

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

In [ ]:
estimators = [
    ("rf", RandomForestClassifier(n_estimators=10, random_state=42)),
    ("svr", make_pipeline(StandardScaler(), LinearSVC(random_state=42)))
]

clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression()
)

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42
)

clf.fit(X_train, y_train).score(X_test, y_test)